#SLEAP_AI.ipnyb

#Description
---------
This notebook runs SLEAP.ai in the colab Virtual Machine.

It uses the power of NVIDIA Tesla T4 GPU to take the best out of SLEAP running it's deep learning neural networks faster.

The system used is condacolab since it helps bypass default python limitations from colab

SLEAP is run inside an environment where it has the necessary drivers.

Run order

1.   Choose the Paths.
2.   Prepare the environment
3.   Run SLEAP
4.   Output the predictions



SLEAP.ai DOI 🔗 https://doi.org/10.1038/s41592-022-01426-1

Moita Lab 🔗 https://moitalab.org/




#Define Paths

In [1]:
import os
from ipyfilechooser import FileChooser
from IPython.display import display, clear_output
import ipywidgets as widgets
from google.colab import drive

# Mount Google Drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# --- INTERFACE CONFIGURATION ---

# 1. Selector for the Experience Folder (Root)
chooser_exp = FileChooser('/content/drive/MyDrive/')
chooser_exp.title = '<b>1. Select Experience Folder (Root)</b>'
chooser_exp.show_only_dirs = True

# 2. Selector for the SLEAP Model
chooser_model = FileChooser('/content/drive/MyDrive/')
chooser_model.title = '<b>2. Select SLEAP Model (.json or folder)</b>'

# Confirmation Button
save_button = widgets.Button(
    description='Confirm and Configure Folders',
    button_style='success',
    icon='check',
    layout=widgets.Layout(width='350px')
)

output_log = widgets.Output()

def on_confirm_clicked(b):
    with output_log:
        clear_output()

        exp_root = chooser_exp.selected_path
        model_path = chooser_model.selected

        if not exp_root or not model_path:
            print("❌ Error: Please select both the Experience Folder and the Model!")
            return

        # Moita Lab Hierarchy
        crop_raw = os.path.join(exp_root, "PostProcessing", "CropRaw")
        arenas = os.path.join(exp_root, "PostProcessing", "Arenas")
        pose_folder = os.path.join(exp_root, "PostProcessing", "Pose")

        # Subfolder verification
        if not os.path.exists(crop_raw) or not os.path.exists(arenas):
            print(f"❌ Error: Could not find the required structure in:\n{exp_root}")
            print("Please ensure 'PostProcessing/CropRaw' and 'PostProcessing/Arenas' exist.")
            return

        # Create Pose folder
        if not os.path.exists(pose_folder):
            os.makedirs(pose_folder)
            print(f"✅ Pose folder created: {pose_folder}")
        else:
            print(f"📂 Pose folder already exists.")

        # Save paths for the Bash environment
        with open('/content/sleap_paths.env', 'w') as f:
            f.write(f'export VIDEO_FOLDER="{crop_raw}"\n')
            f.write(f'export ARENAS_FOLDER="{arenas}"\n')
            f.write(f'export OUTPUT_FOLDER="{pose_folder}"\n')
            f.write(f'export MODEL_PATH="{model_path}"\n')

        print("-" * 40)
        print("🚀 PIPELINE SYNCED AND READY!")
        print(f"Experience: {os.path.basename(exp_root)}")
        print(f"Videos in: .../CropRaw")
        print(f"Images in: .../Arenas")
        print("-" * 40)

save_button.on_click(on_confirm_clicked)

display(chooser_exp, chooser_model, save_button, output_log)

Mounted at /content/drive


FileChooser(path='/content/drive/MyDrive', filename='', title='<b>1. Selecionar Experience Folder (Root)</b>',…

FileChooser(path='/content/drive/MyDrive', filename='', title='<b>2. Selecionar Modelo SLEAP (.json ou pasta)<…

Button(button_style='success', description='Confirmar e Configurar Pastas', icon='check', layout=Layout(width=…

Output()

#Enviroment preparation

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Create a dedicated output folder

!pip install -q condacolab
import condacolab
condacolab.install() # Runtime will restart

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:08
🔁 Restarting kernel...


In [3]:
!mamba create -y -n sleap_env -c conda-forge -c nvidia -c sleap -c anaconda \
    python=3.7 \
    sleap=1.3.4 \
    tensorflow=2.7.0 \
    cudatoolkit=11.3.1 \
    cudnn=8.2.1.32 \
    pandas=1.3.5 \
    scipy=1.7.3 \
    matplotlib=3.5.3


Looking for: ['python=3.7', 'sleap=1.3.4', 'tensorflow=2.7.0', 'cudatoolkit=11.3.1', 'cudnn=8.2.1.32', 'pandas=1.3.5', 'scipy=1.7.3', 'matplotlib=3.5.3']

[+] 0.0s
nvidia/linux-64 (check zst)                        Checked  0.1s
[+] 0.0s
nvidia/noarch (che..  ⣾  nvidia/noarch (check zst)                         
[+] 0.0s
sleap/linux-64 (ch..  ⣾  [+] 0.1s
sleap/linux-64 (ch..  ⣾  [+] 0.2s
sleap/linux-64 (ch..  ⣾  [+] 0.3s
sleap/linux-64 (ch..  ⣾  sleap/linux-64 (check zst)                        
[+] 0.0s
[+] 0.1s
sleap/noarch (chec..  ⣾  [+] 0.2s
sleap/noarch (chec..  ⣾  sleap/noarch (check zst)                          
[+] 0.0s
anaconda/linux-64 ..  ⣾  anaconda/linux-64 (check zst)                     
[+] 0.0s
anaconda/noarch (c..  ⣾  anaconda/noarch (check zst)                       
[+] 0.0s
conda-forge/linux-64  ⣾  
conda-forge/noarch    ⣾  
nvidia/linux-64       ⣾  
nvidia/noarch         ⣾  
sleap/linux-64        ⣾  [+] 0.1s
conda-forge/linux-64   1%
conda-forge/noarch    ⣾  
n

In [4]:
%%bash

# Test if th gpu is available
source activate sleap_env
python -c "import sleap; print('SLEAP Version Check:', sleap.__version__)"
python -c "import tensorflow as tf; print('Num GPUs Available: ', len(tf.config.list_physical_devices('GPU')))"

INFO:matplotlib.font_manager:generated new fontManager
SLEAP Version Check: 1.3.4
Num GPUs Available:  1


2026-02-18 22:50:18.939164: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2026-02-18 22:50:19.075354: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2026-02-18 22:50:19.078029: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


#Running


In [1]:
%%bash
source activate sleap_env
export MPLBACKEND=Agg
source /content/sleap_paths.env

# Start processing
for video in "$VIDEO_FOLDER"/*.avi; do
    [ -e "$video" ] || continue
    filename=$(basename "$video")

    # Logic: remove '_crop.avi' to find the corresponding arena .png name
    # We use a clear bash variable to pass to the python environment
    current_basename="${filename%_crop.avi}"

    echo "------------------------------------------------"
    echo "PROCESSING: $filename"
    echo "BASE NAME: $current_basename"

    # 1. Run SLEAP (Inference)
    sleap-track "$video" \
        -m "$MODEL_PATH" \
        -o "$OUTPUT_FOLDER/${filename}.predictions.slp" \
        --no-gui

    # 2. Python Script for Post-Processing and Normalization
    # We pass variables via environment arguments to avoid NameErrors
    export CUR_FILE="$filename"
    export CUR_BASE="$current_basename"

    python - <<EOF
import sleap
import pandas as pd
import os
import numpy as np
from PIL import Image

# Retrieve paths from environment
arenas_path = os.environ['ARENAS_FOLDER']
output_path = os.environ['OUTPUT_FOLDER']
video_filename = os.environ['CUR_FILE']
basename = os.environ['CUR_BASE']

slp_file = os.path.join(output_path, f"{video_filename}.predictions.slp")
csv_output = os.path.join(output_path, f"{basename}_pose.csv")
png_file = os.path.join(arenas_path, f"{basename}.png")

# Node name mapping for Moita Lab / Bonsai standard
node_mapping = {
    'L': 'Left', 'R': 'Right', 'H': 'Head', 'Trx': 'Thorax',
    'Abd': 'Abdomen', 'Lw': 'LeftWing', 'Rw': 'RightWing', 'T': 'Top'
}

if os.path.exists(slp_file):
    # Attempt to get real resolution from the arena image
    if os.path.exists(png_file):
        with Image.open(png_file) as img:
            w, h = img.size
        print(f"-> Arena resolution detected: {w}x{h}")
    else:
        print(f"-> WARNING: {png_file} not found. Using 1.0 scale (pixels).")
        w, h = 1.0, 1.0

    labels = sleap.load_file(slp_file)
    data = []

    for frame in labels:
        for inst in frame.instances:
            row = {'FrameIndex': frame.frame_idx}
            for node in labels.skeleton.nodes:
                b_name = node_mapping.get(node.name, node.name)
                pt = inst[node.name]

                if pt is not None:
                    row[f'{b_name}.Position.X'] = pt.x / w
                    row[f'{b_name}.Position.Y'] = pt.y / h
                    row[f'{b_name}.Confidence'] = pt.score
                else:
                    row[f'{b_name}.Position.X'] = np.nan
                    row[f'{b_name}.Position.Y'] = np.nan
                    row[f'{b_name}.Confidence'] = 0.0
            data.append(row)

    df = pd.DataFrame(data)
    if not df.empty:
        parts = ['Left', 'Right', 'Top', 'Head', 'Thorax', 'Abdomen', 'LeftWing', 'RightWing']
        cols_order = ['FrameIndex']
        for p in parts:
            cols_order.extend([f'{p}.Position.X', f'{p}.Position.Y', f'{p}.Confidence'])

        df = df.reindex(columns=[c for c in cols_order if c in df.columns])
        df = df.sort_values('FrameIndex')

        # Save CSV with explicit "NaN" string
        df.to_csv(csv_output, index=False, na_rep="NaN")
        print(f"-> Success: {os.path.basename(csv_output)} generated.")
    else:
        print("-> Warning: No instances detected.")
else:
    print(f"-> Error: File {slp_file} not found.")
EOF

done
echo "================================================"
echo "PIPELINE COMPLETE!"

------------------------------------------------
A PROCESSAR: MF-LC6_X_CS_ChS(Retinal)-1Loom_19Opto(15mWcm2)-Female-4days-FH2-CamA-2025-08-15T14_30_05_fly0_crop.avi
BASE NAME: MF-LC6_X_CS_ChS(Retinal)-1Loom_19Opto(15mWcm2)-Female-4days-FH2-CamA-2025-08-15T14_30_05_fly0
Started inference at: 2026-02-18 22:50:36.077821
Args:
{
│   'data_path': '/content/drive/.shortcut-targets-by-id/1EvQfnCVnY3B5N4bSdunnJEZzZHMVxaoS/Matheus_e_Rodrigo/colab_bonsai/PostProcessing/CropRaw/MF-LC6_X_CS_ChS(Retinal)-1Loom_19Opto(15mWcm2)-Female-4days-FH2-CamA-2025-08-15T14_30_05_fly0_crop.avi',
│   'models': [
│   │   '/content/drive/.shortcut-targets-by-id/1EvQfnCVnY3B5N4bSdunnJEZzZHMVxaoS/Matheus_e_Rodrigo/SLEAP/NewDataBase/model/251125_130445.single_instance.n=3283/'
│   ],
│   'frames': '',
│   'only_labeled_frames': False,
│   'only_suggested_frames': False,
│   'output': '/content/drive/.shortcut-targets-by-id/1EvQfnCVnY3B5N4bSdunnJEZzZHMVxaoS/Matheus_e_Rodrigo/colab_bonsai/PostProcessing/Pose/MF-LC6_X_C

2026-02-18 22:50:36.183816: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2026-02-18 22:50:36.194634: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2026-02-18 22:50:36.197109: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2026-02-18 22:50:37.385346: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags